In [1]:
from sqlalchemy import create_engine
import sys
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
localhost = {'user': 'postgres', 'password': 'postgres', 'host': 'localhost', 'port': 5432, 'db': 'fiadb'}
params = 'postgresql://{0}:{1}@{2}:{3}/{4}'
engine = create_engine(params.format(localhost['user'], localhost['password'], localhost['host'], localhost['port'], localhost['db']))
select_query = """
select a.tsr, a.tsr_predicted as glm_pred,
b.tsr_predicted as rf_pred,
c.tsr_predicted as grnn_pred,
d.tsr_predicted as ffnn_pred
FROM predictor.glm a, predictor.random_forest b, predictor.grnn c, predictor.ffnn d
where a.grid_id = b.grid_id
and a.grid_id = c.grid_id
and a.grid_id = d.grid_id
"""
pred_results = pd.read_sql(select_query, engine)

In [3]:
pred_results.head()

,tsr,glm_pred,rf_pred,grnn_pred,ffnn_pred
0,1.0,4.628603,1.820770,2.803938,2.55672
1,18.0,17.476499,19.074224,19.274464,15.80820
2,16.0,19.603017,19.886939,21.177705,22.03830
3,36.0,46.540524,35.683091,38.381778,40.23290
4,27.0,34.607843,34.882035,36.119813,37.65060


In [15]:
pred_results.shape

(15310, 5)

In [5]:
from scipy.stats import wilcoxon
from statistics import median

In [6]:
glm_w, glm_p = wilcoxon(pred_results['tsr'].tolist(), pred_results['glm_pred'].tolist())

In [7]:
rf_w, rf_p = wilcoxon(pred_results['tsr'].tolist(), pred_results['rf_pred'].tolist())

In [8]:
grnn_w, grnn_p = wilcoxon(pred_results['tsr'].tolist(), pred_results['grnn_pred'].tolist())

In [9]:
ffnn_w, ffnn_p = wilcoxon(pred_results['tsr'].tolist(), pred_results['ffnn_pred'].tolist())

In [10]:
print(median(pred_results['tsr'].tolist()), median(pred_results['glm_pred'].tolist()),
     median(pred_results['rf_pred'].tolist()), median(pred_results['grnn_pred'].tolist()),
     median(pred_results['ffnn_pred'].tolist()))

12.0 12.782564109029 12.5873510233424 12.71703548197955 12.4872


In [11]:
print(glm_w, rf_w, grnn_w, ffnn_w)

55122841.0 57574340.0 57483148.0 57211620.5


In [14]:
print(glm_p, rf_p, grnn_p, ffnn_p)

1.9737322031494704e-10 0.06001473650742681 0.0406154561673107 0.9036441874952
